In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy
    !pip install --no-deps git+https://github.com/mmathew23/unsloth-zoo.git@t4mixed
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
!pip install --upgrade datasets

In [ ]:
%env UNSLOTH_RETURN_LOGITS=1 # Run this to disable CCE since it is not supported for CPT

%env TRANSFORMERS_VERBOSITY=info

env: UNSLOTH_RETURN_LOGITS=1 # Run this to disable CCE since it is not supported for CPT
env: TRANSFORMERS_VERBOSITY=info


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 512
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",

                      "embed_tokens", "lm_head",],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True,
    loftq_config = None,
)

In [ ]:
from textblob import TextBlob
import nltk
import os
from google.colab import drive


nltk.download('punkt_tab')

drive.mount('/content/drive')

folder_path = "path"


sentences = []


for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        blob = TextBlob(text)
        sentences.extend(blob.sentences)


print(f"Collected {len(sentences)} total sentences.")
print(sentences[:1])


In [ ]:

chunks = []
for s in range(len(sentences) - 3):
  chunks.append(" ".join([str(sent) for sent in sentences[s:s+3]]))

chunks[900]

'Blow trees on spots with no geese\nKnow these cops is so hungry\nThe gross g’s all fuss and penitentiaries\nFrustrated the only way to bust is mentally\nFor ghettoes are metal, stayed bopping the instrumentals\nWhen cops is cock blocking on front of spots\nWe off top switching the pedals, metal in front of our wheels\nSo aliens could sit in the middle, got caught in the middle\nWith a little bit of witticism, I’m in the city of villains\nSpazzing crouching tigers and hiden dragons\nNiggas are ghost spitamatic, we flow spitamatics\nFor the head piece to get attractive, fern and red leaves\nTo the cabbage, learning that these niggas acting\nWe gonna keep rapping, til the streets cease the madness\nListening to beats clapping to be happy\nStayed so nappy, combs’ll hold the gat piece\nJust to get through, feeling like we gotta spit missiles\nJust to get you to feel it in a physical\nReciprocate lyricals and fashions to bash in\nYour cranial cabinet, metaphors been words\nWe come hands to 

In [ ]:

import pandas as pd
df = pd.DataFrame(chunks, columns=['text'])
df

,text
0,Fanaticism\n==========\nCross my fingers for h...
1,"Objects may be nearer\nIn debt, see life clear..."
2,"pens, spitting phlegm and bloody gems\nCorner ..."
3,"splitting factions, bagging up fractions\nMovi..."
4,""", give me an L and V8\nTo see straight throug..."
...,...
1182,What does? Kingship\nI haven't noticed it when...
1183,Kingship\nI haven't noticed it when I bleed bi...
1184,bigfakelaugh\n[Intro]\nYou mean today? No n-wo...
1185,"No n-words, no Irish\nShopkeeper pointed to th..."


In [ ]:

from datasets import Dataset
dataset = Dataset.from_pandas(df)
dataset[900]

{'text': 'Blow trees on spots with no geese\nKnow these cops is so hungry\nThe gross g’s all fuss and penitentiaries\nFrustrated the only way to bust is mentally\nFor ghettoes are metal, stayed bopping the instrumentals\nWhen cops is cock blocking on front of spots\nWe off top switching the pedals, metal in front of our wheels\nSo aliens could sit in the middle, got caught in the middle\nWith a little bit of witticism, I’m in the city of villains\nSpazzing crouching tigers and hiden dragons\nNiggas are ghost spitamatic, we flow spitamatics\nFor the head piece to get attractive, fern and red leaves\nTo the cabbage, learning that these niggas acting\nWe gonna keep rapping, til the streets cease the madness\nListening to beats clapping to be happy\nStayed so nappy, combs’ll hold the gat piece\nJust to get through, feeling like we gotta spit missiles\nJust to get you to feel it in a physical\nReciprocate lyricals and fashions to bash in\nYour cranial cabinet, metaphors been words\nWe come 

Print out 5 stories from the dataset

In [ ]:
for row in dataset[:5]["text"]:
    print("=========================")
    print(row)

Fanaticism
Cross my fingers for heaven and a burn a chalice with brethren
Three-six-five, twenty-four-seven, hold cigars like 7-11
Jars of that seven-four-seven, coming to America, Johnny Dangerous
Flex mic stainless to spit at close ranges
Who these strangers in the mirror? Objects may be nearer
In debt, see life clearer, if drugs is terror, call me Pol Pot
Red rags and ? pens, spitting phlegm and bloody gems
Corner store hymns, Russian Roulette among friends
Forgotten Gods guard you now, Gunzu mile, don't fuck around
Piru's will paint the town, incoming, hit the ground
Have a brown bag beer and a frown
I drank from Mommas tears, came up fighting my peers
Shots busted and no need to fear
The bullet that kills you is the one you ain't hear
Kalashnikovs and spears, four-hundred years
I been ready, pass the machete on some Mario Andrede
Gimme some gas, bags of diesel, powder to the people
Courtesy of pigeons waving Desert Eagles
Crumb snatching like "Sorry Miss Jackson, it's that real"
S

Tokenize it!

In [ ]:
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    return { "text" : [example + EOS_TOKEN for example in examples["text"]] }
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 8,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,

        warmup_ratio = 0.1,
        num_train_epochs = 3,

        learning_rate = 2e-5,
        embedding_learning_rate = 5e-6,

        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,

        optim = "adamw_torch_fused",
        weight_decay = 0.00,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
)
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
import textwrap
max_print_width = 50


FastLanguageModel.for_inference(model)

inputs = tokenizer(
[
    "Foreman we rush"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    max_new_tokens = 256,
    use_cache = True,
    do_sample=True,
    temperature=1.0,
    top_k = 50,
)

outputs = model.generate(**generation_kwargs)
generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)

for text in generated_text:
    wrapped_text = textwrap.fill(text, width=max_print_width)
    print(wrapped_text)

I am gay, but he is straight and that's that. But
that's that, we're still best friends, though Dear
friends, you know what it is Dear friends, it's no
use going half I'm done trying to reason with you
Congratulations on your 6 year run I have nothing
but respect But when I look in the mirror See my
reflection and I'm just not having fun


## Save the model as a GGUF

hopefully...

In [ ]:
model.push_to_hub("path", token = "token")
tokenizer.push_to_hub("path", token = "token")